In [14]:
%matplotlib inline

In [15]:
print(__doc__)
import numpy as np
import json
from sklearn.datasets import load_boston
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from skopt import gp_minimize
import pickle
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.linear_model import ElasticNet
from skopt.plots import plot_convergence
from matplotlib import pyplot as plt
from skopt.space import Real, Integer
from skopt.utils import use_named_args

Automatically created module for IPython interactive environment


In [4]:
BIDS_path = 'C:\\Users\\ICN_admin\\Documents\\BIDS\\'
subject_id = ['000', '001', '004', '005', '006', '007', '008', '009', '010', '013', '014']
out_path = 'C:\\Users\\ICN_admin\\Documents\\ECOGVSTN\\dat\\'

In [5]:
def append_time_dim(arr, y_, time_stamps):
    """
    apply added time dimension for the data array and label given time_stamps
    """
    time_arr = np.zeros([arr.shape[0]-time_stamps, int(time_stamps*arr.shape[1])])
    for time_idx, time_ in enumerate(np.arange(time_stamps, arr.shape[0])):
        for time_point in range(time_stamps):
            time_arr[time_idx, time_point*arr.shape[1]:(time_point+1)*arr.shape[1]] = arr[time_-time_point,:]
    return time_arr, y_[time_stamps:]

In [17]:
# The list of hyper-parameters we want to optimize. For each one we define the
# bounds, the corresponding scikit-learn parameter name, as well as how to
# sample values from that dimension (`'log-uniform'` for the learning rate)
n_features = 40
space  = [Integer(1, 100, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='learning_rate'),
          Integer(10**0, 10**3, "log-uniform", name='n_estimators'), 
          Real(10**0, 10**1, "uniform", name="gamma")]

space_LM = [Real(0, 1, "uniform", name='alpha'),
           Real(0, 1, "uniform", name='l1_ratio')]
           #Integer(1000, 5000, name='max_iter')]

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result. The metric name must not contain a
    # colon (:) or a space since preds are margin(before logistic
    # transformation, cutoff at 0)
    
    r2 = metrics.r2_score(labels, preds)
    
    if r2 < 0:
        r2 = 0
    
    return 'r2', r2

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set
# scikit-learn estimator parameters
@use_named_args(space_LM)
def objective(**params):
    print(params)
    
    params_ = {'alpha': params["alpha"],
         'l1_ratio': params['l1_ratio']}
         #'max_iter': params["max_iter"]}
    
    reg.set_params(**params_)
    cv_res = cross_val_score(reg, X_time, y_time, cv=3, n_jobs=59, scoring="r2")
    
    cv_res[np.where(cv_res < 0)[0]] = 0
    print(np.mean(cv_res))
    return -np.mean(cv_res)

In [19]:
for subject in subject_id:
    with open('dat\\sub_'+subject+'_patient_concat.json') as f:
        data = json.load(f)
        channels = list(data.keys())

        d_xgb = {}

        for ch in channels:
            X = np.array(data[ch]['data']).T
            y_ = np.array(data[ch]['true_movements'])
            mov_ch = data[ch]['mov_ch']

            d_xgb[ch] = {}

            for y_idx in range(y_.shape[0]):
                print('subject: '+str(subject)+' channel: '+str(ch)+' mov: '+str(mov_ch[y_idx]))
                y = y_[y_idx,:]

                X_time, y_time = append_time_dim(X, y, 5)

                reg = ElasticNet()
                res_gp = gp_minimize(objective, space_LM, n_calls=20)
                #plot_convergence(res_gp)
                #plt.title("subject: "+str(subject) + " channel: " + str(ch) + " mov_ch: "+str(mov_ch[y_idx]))
                #plt.show()
                d_xgb[ch][mov_ch[y_idx]] = res_gp
        with open('Redo_lmelasticnet_sub_'+subject+'.p', 'wb') as handle:
            pickle.dump(d_xgb, handle, protocol=pickle.HIGHEST_PROTOCOL)


subject: 000 channel: ECOG_RIGHT_0 mov: MOV_RIGHT
{'alpha': 0.1567049454455516, 'l1_ratio': 0.6735738806413533}
0.062147357744537945
{'alpha': 0.6119062408920584, 'l1_ratio': 0.283305372385054}
0.010202441352696212
{'alpha': 0.03128762038119438, 'l1_ratio': 0.7406614722594955}
0.1429332423012084
{'alpha': 0.04623508810558375, 'l1_ratio': 0.5226103857765205}
0.14128714149668897
{'alpha': 0.6542083052165242, 'l1_ratio': 0.535401011883904}
0.0
{'alpha': 0.5371055355377755, 'l1_ratio': 0.29770109149099205}
0.018797055350885866
{'alpha': 0.034546917140022455, 'l1_ratio': 0.011456088795096743}
0.15411348662824045
{'alpha': 0.9452422888465626, 'l1_ratio': 0.9411049138982422}
0.0
{'alpha': 0.9622074732738755, 'l1_ratio': 0.9118988066296517}
0.0
{'alpha': 0.5781873256730139, 'l1_ratio': 0.09445167282735448}
0.09820231744279447
{'alpha': 0.2893100472399871, 'l1_ratio': 0.0}
0.14409093114211122
{'alpha': 0.0, 'l1_ratio': 1.0}
0.1539419429278641
{'alpha': 0.0, 'l1_ratio': 0.2770465175178188}
0.153

C:\Users\ICN_admin\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


0.6657421798304208
{'alpha': 0.05881837053446104, 'l1_ratio': 0.0}
0.6545336135096608
{'alpha': 0.0, 'l1_ratio': 1.0}
0.6657421798304208
{'alpha': 0.0, 'l1_ratio': 0.6181796568149012}
0.6657421798304208
{'alpha': 0.0, 'l1_ratio': 0.26707679196241413}
0.6657421798304208
{'alpha': 0.0, 'l1_ratio': 0.8314885473985896}
0.6657421798304208
{'alpha': 1.0, 'l1_ratio': 0.0}
0.4832962320835888
{'alpha': 0.0, 'l1_ratio': 0.4440769166751074}
0.6657421798304208
{'alpha': 0.04173583321993518, 'l1_ratio': 1.0}
0.6143944667283228
subject: 000 channel: ECOG_RIGHT_1 mov: MOV_RIGHT
{'alpha': 0.30245790920142096, 'l1_ratio': 0.021561193087141776}
0.1552487585821032
{'alpha': 0.6709972228714005, 'l1_ratio': 0.9422774983941681}
0.0
{'alpha': 0.6178367824870232, 'l1_ratio': 0.09374645203432287}
0.10028726946585997
{'alpha': 0.8416556719835262, 'l1_ratio': 0.5722992951143925}
0.0
{'alpha': 0.6807311454738689, 'l1_ratio': 0.8899163772258115}
0.0
{'alpha': 0.8952871676163974, 'l1_ratio': 0.6698664898099123}
0.0

C:\Users\ICN_admin\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


{'alpha': 0.0, 'l1_ratio': 0.0}
0.17409992970561236
subject: 000 channel: ECOG_RIGHT_1 mov: MOV_LEFT
{'alpha': 0.25125035358980824, 'l1_ratio': 0.4957225044876912}
0.40749304488546206
{'alpha': 0.8872145987725925, 'l1_ratio': 0.47771269951079165}
0.004035770232639457
{'alpha': 0.6161091954878636, 'l1_ratio': 0.052235531518583414}
0.4291281037472577
{'alpha': 0.14314120336487127, 'l1_ratio': 0.14219455581147436}
0.5390888981612932
{'alpha': 0.09305926570124902, 'l1_ratio': 0.40856972345010534}
0.5302879153640916
{'alpha': 0.12534866917396273, 'l1_ratio': 0.42259662788826957}
0.4930664423312612
{'alpha': 0.00939516658841444, 'l1_ratio': 0.7499761300225288}
0.6302525197219292
{'alpha': 0.15436783411026891, 'l1_ratio': 0.867282700252695}
0.4303175021768064
{'alpha': 0.3743531835542932, 'l1_ratio': 0.2976949309308003}
0.3978991553242248
{'alpha': 0.30265335688381184, 'l1_ratio': 0.8392888125646644}
0.2591936876625985
{'alpha': 0.0, 'l1_ratio': 0.8461849282599881}
0.6316482529484491
{'alpha'

...

0.020353944382014322
{'alpha': 0.0067278740424561185, 'l1_ratio': 0.0010574493722553815}
0.020214307305459367
{'alpha': 0.0022335534135915496, 'l1_ratio': 0.0019845663709804344}
0.02525739351018741
{'alpha': 0.005573289642019975, 'l1_ratio': 0.9822562082757065}
0.008652519359512797
{'alpha': 0.005478136137536783, 'l1_ratio': 0.005011649948084674}
0.02162557671437008
{'alpha': 0.0, 'l1_ratio': 0.0}


C:\Users\ICN_admin\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


0.020353944382014322
{'alpha': 0.9941407212962301, 'l1_ratio': 0.9960496830272205}
0.0
{'alpha': 0.001033342986224928, 'l1_ratio': 0.9681051462848551}
0.024115752194919793
subject: 014 channel: STN_RIGHT_3 mov: MOV_LEFT
{'alpha': 0.020371453456767167, 'l1_ratio': 0.06427821270881164}
0.0
{'alpha': 0.0658771996280494, 'l1_ratio': 0.768454418797984}
0.0
{'alpha': 0.8486103823155555, 'l1_ratio': 0.09092908579924742}
0.0
{'alpha': 0.9951516045293604, 'l1_ratio': 0.7730797369468857}
0.0
{'alpha': 0.8888808856302778, 'l1_ratio': 0.8529819520602285}
0.0
{'alpha': 0.9379962994990478, 'l1_ratio': 0.7107489210512514}
0.0
{'alpha': 0.0019490555353625099, 'l1_ratio': 0.44236436092141707}
0.0
{'alpha': 0.23613468371477514, 'l1_ratio': 0.12295439684122514}
0.002685986766810089
{'alpha': 0.47279771874384335, 'l1_ratio': 4.037288271607765e-05}
0.0038291777775736335
{'alpha': 0.5344554395338562, 'l1_ratio': 0.7021002685396793}
0.0
{'alpha': 0.00013869362857121817, 'l1_ratio': 0.9916660883980511}
0.0
{'

C:\Users\ICN_admin\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


0.003382166594793327
{'alpha': 0.9989948287102051, 'l1_ratio': 0.9951358018840434}
0.0
